> Chap 8.3: Discrete Fourier Transform

In [32]:
using LinearAlgebra, FFTW

Shift Operator B가 존재할 떄,

- 직교 행렬orthogonal
- cyclic shift operator($BS_n = S_{n-1}$) $\to$ $S_{n-1}$은 vector space components

$\star$ 시계열의 back shift operator 로 생각할 수 있고, foward shift operator도 가능하다. 

$\star$ cyclic operator이어야 하는 이유? 책의 정의 이용 및 back/forward shift operator는 고유분해 안 될 수도.

이 행렬을 고유분해(full rank)하여 나온 고유값과 고유벡터가 존재한다.

In [45]:
B= [0 0 0 0 1
    1 0 0 0 0 
    0 1 0 0 0
    0 0 1 0 0
    0 0 0 1 0] # cyclic shift operator B
B'B # matrix B is orthogonal

5×5 Matrix{Int64}:
 1  0  0  0  0
 0  1  0  0  0
 0  0  1  0  0
 0  0  0  1  0
 0  0  0  0  1

In [46]:
s = [1,2,3,4,5]
B*s # matrix B is a cyclic shift operator

5-element Vector{Int64}:
 5
 1
 2
 3
 4

In [47]:
B^2*s

5-element Vector{Int64}:
 4
 5
 1
 2
 3

이 고유값$\lambda$, 고유벡터$\psi$가 존재한다면 B는 $DFT^* \Lambda DFT$로 표현 가능핟하다.

- $DFT^*$
    - conjugate matrix
    - $\psi$인데 DFT로 표현 $\to$ 그래프 도메인으로 확장이 가능하기 때문
    
여기서 $DFT^*$는 $\psi^*_k = DFT_k = \frac{1}{\sqrt{N}} \begin{bmatrix} 1 \\ \dots \\ e^{-j\frac{2\pi}{N}(N-1)k} \end{bmatrix}$로서 표현($\in C^N$ 길이가 $N$인 vector(복소수))

- unitary and symmetric
    - unitary $\to$ complex space에서 정규직교기저를 이루고, $A(A^*)^\top = I, \psi^{-1} = \psi^*, \psi^* \psi = \psi \psi^* = I$
- 위 $k$개의 벡터들은 `spectral components` 이다.
- the complex exponential sinusodal functions

여기서 $\lambda$는 the `frequencies of the signal` 로서 정의될 수 있다.

- 특징
    - distinct
    - positive
    - equally spaced
    - increasing from $0$ ro $\frac{N-1}{N}$

In [48]:
λ, ψ = eigen(B)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
5-element Vector{ComplexF64}:
 -0.8090169943749472 - 0.5877852522924725im
 -0.8090169943749472 + 0.5877852522924725im
 0.30901699437494734 - 0.9510565162951536im
 0.30901699437494734 + 0.9510565162951536im
  0.9999999999999998 + 0.0im
vectors:
5×5 Matrix{ComplexF64}:
  0.138197+0.425325im   0.138197-0.425325im  …  0.447214+0.0im
 -0.361803-0.262866im  -0.361803+0.262866im     0.447214+0.0im
  0.447214-0.0im        0.447214+0.0im          0.447214+0.0im
 -0.361803+0.262866im  -0.361803-0.262866im     0.447214+0.0im
  0.138197-0.425325im   0.138197+0.425325im     0.447214+0.0im

In [49]:
B = ψ * Diagonal(λ) * ψ'

5×5 Matrix{ComplexF64}:
    2.498e-16-9.67158e-18im  …           1.0+1.81709e-18im
          1.0+2.1793e-18im       5.55112e-16-1.09573e-17im
 -3.88578e-16-7.89355e-18im     -3.21902e-16+8.57473e-18im
 -4.16334e-16-8.06149e-18im       -4.996e-16-8.9293e-18im
  2.99888e-16+1.53977e-17im      3.99189e-16+1.42498e-17im

In [50]:
DFT = ψ'

5×5 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
  0.138197-0.425325im  -0.361803+0.262866im  …  0.138197+0.425325im
  0.138197+0.425325im  -0.361803-0.262866im     0.138197-0.425325im
 -0.361803-0.262866im  -0.361803+0.262866im     0.138197-0.425325im
 -0.361803+0.262866im  -0.361803-0.262866im     0.138197+0.425325im
  0.447214-0.0im        0.447214-0.0im          0.447214-0.0im

$B = \psi \Lambda \psi^{-1}$

- $\psi^H := DFT$ 이렇게 정의한다면 F의 고유벡터의 conjugate
- $F$ $\to$ $BF = I$
- $\psi \Lambda \psi^H  F = I$
- 만약, $F$가 $\psi^H\Lambda^{-1}\psi$라면, $\psi \Lambda \psi^H\psi^H\Lambda \psi = I$
- 따라서 $F$는 $\psi^{-1} \Lambda^{-1} \psi$로 고유분해

In [51]:
x = [1,2-im,-im,-1+2im]

4-element Vector{Complex{Int64}}:
  1 + 0im
  2 - 1im
  0 - 1im
 -1 + 2im

In [52]:
_DFT = reshape([i*j for i in 0:3 for j in 0:3], (4,4))
_DFT

4×4 Matrix{Int64}:
 0  0  0  0
 0  1  2  3
 0  2  4  6
 0  3  6  9

In [53]:
f = x -> exp(-im * (2π/4) * x)
DFT = _DFT .|> f

4×4 Matrix{ComplexF64}:
 1.0-0.0im           1.0-0.0im          …           1.0-0.0im
 1.0-0.0im   6.12323e-17-1.0im             -1.83697e-16+1.0im
 1.0-0.0im          -1.0-1.22465e-16im             -1.0-3.67394e-16im
 1.0-0.0im  -1.83697e-16+1.0im              5.51091e-16-1.0im

In [55]:
DFT * x

4-element Vector{ComplexF64}:
                   2.0 + 0.0im
   -1.9999999999999998 - 2.0000000000000004im
 8.881784197001252e-16 - 1.9999999999999998im
    3.9999999999999987 + 4.000000000000001im

In [54]:
fft(x)

4-element Vector{ComplexF64}:
  2.0 + 0.0im
 -2.0 - 2.0im
  0.0 - 2.0im
  4.0 + 4.0im

DFT의 두 번쨰 정의

- 복소수 sequence $\{x_n\}$을 `규칙`에 따라 $\{X_k\}$로 변환하는 것
- 규칙: $x_k = \sum^{N-1}_{n=0} x_n e^{-i\frac{2\pi}{N}kn}$
    - 특히, $k=0$이면 $X_0 = \sum^{N-1}{n=0}x_n$, constant term 이 되어 $\beta_0$의 역할을 한다.

행렬로 표현한다면, $\begin{bmatrix}X_k \\ \dots \end{bmatrix} = DFT = \begin{bmatrix}x_n \\ \dots \end{bmatrix}$

- $x_k = DFT^{-1}X_k$
    - $x_k$ = bias, 관측값
    - $DFT^{-1}$: 설명변수, unitary라 $DFT^{-1} = DFT = DFT^*$, symmetric, orthogonal(설명변수가 독립적이라 다중공선성이 존재하지 않는다.)
        - 다중공선성이 있으면 각 설명변수의 설명이 안 될 수도 있고 그 설명변수를 해석하기도 어려워짐.
    - $X_k = \beta$, codfficient(푸리에 변환의 결과이다)

DFT 행렬의 특징

1. 유니터리unitary 행렬, 즉, $DFT^* = DFT, DFT^*DFT = I$
2. 대칭symmetric 행렬 $\to$ 그렇기 떄문에 이 행렬의 켤레전치는 $i = \sqrt{-1}$ 대신 $i$를 넣은 것과 같음.

- inverse DFT는 $i = -i$를 넣은 행렬, 즉 DFT의 켤레전치 = inverse DFT

$DFT = \frac{1}{\sqrt{N}}\begin{bmatrix} 1 & 1 & 1 & \dots & 1 \\ 1 & e^{-i\frac{2\pi}{N}1} & e^{-i\frac{2\pi}{N}1} & \dots & e^{-i\frac{2\pi}{N}(N-1)} \\ 1 & e^{-i\frac{2\pi}{N}2} & e^{-i\frac{2\pi}{N}4} & \dots & e^{-i\frac{2\pi}{N}(2(N-1)} \\ \dots & \dots & \dots & \dots \\ 1 & e^{-i\frac{2\pi}{N}(N-1)} & e^{-i\frac{2\pi}{N}2(N-1)} & \dots & e^{-i\frac{2\pi}{N}(N-1)^2}\end{bmatrix}$

In [59]:
DFT = (1/√4)*DFT # 위의 정의 충족위해 1/sqrt(4)곱함
DFT'DFT .|> round # 유니터리행렬임을 확인!

4×4 Matrix{ComplexF64}:
  0.0+0.0im  -0.0-0.0im   0.0-0.0im   0.0-0.0im
 -0.0+0.0im   0.0+0.0im  -0.0-0.0im   0.0-0.0im
  0.0+0.0im  -0.0+0.0im   0.0+0.0im  -0.0-0.0im
  0.0+0.0im   0.0+0.0im  -0.0+0.0im   0.0+0.0im